In [1]:
import numpy as np
import pandas as pd

In [2]:
houses = pd.read_csv("train.csv")
print(houses.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [3]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
houses["MSSubClass"].value_counts()

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64

We'll divide the dataset between expansive (> 180000) and non-expensive houses. A classification approach will allow to see which variables are important for our problem.

In [5]:
houses["Expensive"] = 0
houses.loc[houses["SalePrice"]>180000, "Expensive"] = 1

In [6]:
houses["Expensive"].value_counts()

0    896
1    564
Name: Expensive, dtype: int64

In [7]:
pd.crosstab(houses["MSSubClass"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
MSSubClass,,,
20,327,209,536
30,69,0,69
40,3,1,4
45,12,0,12
50,122,22,144
60,64,235,299
70,42,18,60
75,10,6,16
80,46,12,58


Categories 60 and 120 seem to contain more expensive houses, while category 30 contains only non-expensive houses. 30 means old and small houses.

In [8]:
pd.crosstab(houses["MSZoning"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
MSZoning,,,
C (all),10,0,10
FV,22,43,65
RH,14,2,16
RL,643,508,1151
RM,207,11,218
Total,896,564,1460


Category RM seems to contain mostly non-expensive houses. RM means Residential Medium Density. RL means Residential Low Density. We can see that in residential areas where the density is higher, prices in houses tend to be cheaper.

In [9]:
houses.groupby(houses["Expensive"])["LotFrontage"].mean()

Expensive
0    64.954485
1    78.433921
Name: LotFrontage, dtype: float64

Not surprisingly, more expensive houses have more feet of street connected to property.

In [10]:
houses.groupby(houses["Expensive"])["LotArea"].mean()

Expensive
0     8816.839286
1    13217.519504
Name: LotArea, dtype: float64

Same conclusion for the lot area. Bigger is more expensive.

In [11]:
pd.crosstab(houses["Street"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Street,,,
Grvl,4,2,6
Pave,892,562,1454
Total,896,564,1460


The type of road access is not usable for our analysis

In [12]:
pd.crosstab(houses["Alley"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Alley,,,
Grvl,49,1,50
Pave,27,14,41
Total,76,15,91


Alley has too much missing data to be useful (only 91 non-null for 1460 houses)

In [13]:
pd.crosstab(houses["LotShape"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
LotShape,,,
IR1,213,271,484
IR2,13,28,41
IR3,3,7,10
Reg,667,258,925
Total,896,564,1460


Slightly irregular properties tend to be more expensive, which is surprising

In [14]:
pd.crosstab(houses["LandContour"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
LandContour,,,
Bnk,51,12,63
HLS,19,31,50
Low,15,21,36
Lvl,811,500,1311
Total,896,564,1460


HLS and Low seem more expensive, but few houses are impacted by this factor.

In [15]:
pd.crosstab(houses["Utilities"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Utilities,,,
AllPub,895,564,1459
NoSeWa,1,0,1
Total,896,564,1460


Not useful

In [16]:
pd.crosstab(houses["LotConfig"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
LotConfig,,,
Corner,161,102,263
CulDSac,32,62,94
FR2,28,19,47
FR3,1,3,4
Inside,674,378,1052
Total,896,564,1460


Inside might be slightly cheaper, but it's not obvious.

In [17]:
pd.crosstab(houses["LandSlope"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
LandSlope,,,
Gtl,858,524,1382
Mod,32,33,65
Sev,6,7,13
Total,896,564,1460


Most of the houses have gentle slope, so it's not gonna be very discriminant

In [18]:
pd.crosstab(houses["Neighborhood"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Neighborhood,,,
Blmngtn,7,10,17
Blueste,2,0,2
BrDale,16,0,16
BrkSide,51,7,58
ClearCr,7,21,28
CollgCr,52,98,150
Crawfor,21,30,51
Edwards,91,9,100
Gilbert,37,42,79


Neighborhood is important, but it is to be used with a good knowledge of the city.

In [19]:
pd.crosstab(houses["Condition1"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Condition1,,,
Artery,44,4,48
Feedr,68,13,81
Norm,747,513,1260
PosA,1,7,8
PosN,6,13,19
RRAe,11,0,11
RRAn,17,9,26
RRNe,0,2,2
RRNn,2,3,5


Most of the houses are normal, so not very discriminant

In [20]:
pd.crosstab(houses["Condition2"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Condition2,,,
Artery,2,0,2
Feedr,6,0,6
Norm,885,560,1445
PosA,0,1,1
PosN,0,2,2
RRAe,0,1,1
RRAn,1,0,1
RRNn,2,0,2
Total,896,564,1460


Not enough data

In [21]:
pd.crosstab(houses["BldgType"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
BldgType,,,
1Fam,714,506,1220
2fmCon,29,2,31
Duplex,48,4,52
Twnhs,39,4,43
TwnhsE,66,48,114
Total,896,564,1460


Not discriminant

In [22]:
pd.crosstab(houses["HouseStyle"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
HouseStyle,,,
1.5Fin,130,24,154
1.5Unf,14,0,14
1Story,465,261,726
2.5Fin,3,5,8
2.5Unf,9,2,11
2Story,188,257,445
SFoyer,35,2,37
SLvl,52,13,65
Total,896,564,1460


2Story is generally more expensive than 1Story

In [23]:
pd.crosstab(houses["OverallQual"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
OverallQual,,,
1,2,0,2
2,3,0,3
3,20,0,20
4,115,1,116
5,378,19,397
6,277,97,374
7,89,230,319
8,11,157,168
9,0,43,43


More quality = More expensive

In [24]:
pd.crosstab(houses["OverallCond"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
OverallCond,,,
1,1,0,1
2,4,1,5
3,23,2,25
4,54,3,57
5,386,435,821
6,203,49,252
7,155,50,205
8,58,14,72
9,12,10,22


Results are a bit surprising. Low condition houses seem to be cheaper, but they are not very numerous. Less useful than the quality of the materials.

In [27]:
pd.crosstab(houses["YearBuilt"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
YearBuilt,,,
1872,1,0,1
1875,1,0,1
1880,2,2,4
1882,1,0,1
1885,2,0,2
...,...,...,...
2007,8,41,49
2008,1,22,23
2009,2,16,18


In [28]:
houses.groupby(houses["Expensive"])["YearBuilt"].mean()

Expensive
0    1958.851562
1    1990.992908
Name: YearBuilt, dtype: float64

Older houses are cheaper

In [29]:
houses.groupby(houses["Expensive"])["YearRemodAdd"].mean()

Expensive
0    1976.712054
1    1997.819149
Name: YearRemodAdd, dtype: float64

To explore further

In [30]:
pd.crosstab(houses["RoofStyle"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
RoofStyle,,,
Flat,6,7,13
Gable,730,411,1141
Gambrel,7,4,11
Hip,149,137,286
Mansard,4,3,7
Shed,0,2,2
Total,896,564,1460


Might be a bit useful

In [31]:
pd.crosstab(houses["RoofMatl"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
RoofMatl,,,
ClyTile,1,0,1
CompShg,886,548,1434
Membran,0,1,1
Metal,1,0,1
Roll,1,0,1
Tar&Grv,6,5,11
WdShake,0,5,5
WdShngl,1,5,6
Total,896,564,1460


Not useful

In [32]:
pd.crosstab(houses["Exterior1st"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Exterior1st,,,
AsbShng,20,0,20
AsphShn,1,0,1
BrkComm,2,0,2
BrkFace,28,22,50
CBlock,1,0,1
CemntBd,23,38,61
HdBoard,169,53,222
ImStucc,0,1,1
MetalSd,185,35,220


To be used

In [33]:
pd.crosstab(houses["Exterior2nd"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Exterior2nd,,,
AsbShng,19,1,20
AsphShn,3,0,3
Brk Cmn,7,0,7
BrkFace,15,10,25
CBlock,1,0,1
CmentBd,22,38,60
HdBoard,151,56,207
ImStucc,5,5,10
MetalSd,180,34,214


Mostly the same as Exterior1st

In [34]:
pd.crosstab(houses["MasVnrType"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
MasVnrType,,,
BrkCmn,13,2,15
BrkFace,219,226,445
None,639,225,864
Stone,25,103,128
Total,896,556,1452


No veneer is cheaper

In [35]:
houses.groupby(houses["Expensive"])["MasVnrArea"].mean()

Expensive
0     60.393973
1    173.449640
Name: MasVnrArea, dtype: float64

In [36]:
pd.crosstab(houses["ExterQual"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
ExterQual,,,
Ex,3,49,52
Fa,13,1,14
Gd,109,379,488
TA,771,135,906
Total,896,564,1460


Good quality exterior means more expensive house

In [37]:
pd.crosstab(houses["ExterCond"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
ExterCond,,,
Ex,2,1,3
Fa,28,0,28
Gd,104,42,146
Po,1,0,1
TA,761,521,1282
Total,896,564,1460


Not discriminant

In [38]:
pd.crosstab(houses["Foundation"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
Foundation,,,
BrkTil,128,18,146
CBlock,525,109,634
PConc,214,433,647
Slab,23,1,24
Stone,4,2,6
Wood,2,1,3
Total,896,564,1460


Has a significant impact

In [39]:
pd.crosstab(houses["BsmtQual"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
BsmtQual,,,
Ex,11,110,121
Fa,33,2,35
Gd,246,372,618
TA,570,79,649
Total,860,563,1423


Higher basement means more expensive. We have some missing values.

In [40]:
pd.crosstab(houses["BsmtCond"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
BsmtCond,,,
Fa,39,6,45
Gd,27,38,65
Po,2,0,2
TA,792,519,1311
Total,860,563,1423


Not discriminant

In [41]:
pd.crosstab(houses["BsmtExposure"], houses["Expensive"], margins=True, margins_name="Total")

Expensive,0,1,Total
BsmtExposure,,,
Av,105,116,221
Gd,41,93,134
Mn,57,57,114
No,657,296,953
Total,860,562,1422


An exposure means slightly more expensive house